[View in Colaboratory](https://colab.research.google.com/github/msnaru/NaroRecommender/blob/master/negaposi_naro_test.ipynb)

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
!apt-get install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip3 install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [0]:
# データダウンロード

# 解析データのダウンロード
file_id = "1aJD1qK07aWlkP16O8P3-ZphEg48cD-20"
drive_file = drive.CreateFile({'id': file_id})
file = drive_file.GetContentString()

# ネガポジ辞書のダウンロード
dic_id = "1Bc-qeqWdNPvDHDLGj1RB8XPX7y4fL2s9"
dic_file = drive.CreateFile({'id': dic_id})
dic_file.GetContentFile('pn_ja.csv')
pn_df = pd.read_csv("./pn_ja.csv",\
                    sep=':',
                    encoding='shift_jis',
                    names=('Word','Reading','POS', 'PN')
                   )

In [0]:
import MeCab
import pandas as pd
import re

mecab = MeCab.Tagger('')

def get_diclist(file):
  """get_diclistのdocumnent
  textファイルから、BaseFormに関するdicのlistを返す 
  
  Parameters
    ----------
    file : str
    　　対象のテキスト

  Returns
    -------
    diclist : list
        BaseFormをkeyとしたlist
  """
  parsed = mecab.parse(file)
  lines = parsed.split('\n')
  lines = lines[0: -2]

  #解析結果のリストを作成します
  diclist = []
  for word in lines:
      # タブとカンマで区切ったデータを作成します
      data = re.split('\t|,',word)  
      datalist = {'BaseForm':data[7]}
      diclist.append(datalist)
  return(diclist)

wordlist = get_diclist(file)
print(wordlist)

In [0]:
print(get_diclist.__doc__)

In [40]:
# PNTableを単語と極性値のみのdict型に変更
word_list = pn_df['Word'].tolist()
pn_list = pn_df['PN'].tolist()
pn_dict = dict(zip(word_list, pn_list))

# 解析結果のリストからPNTableに存在する単語を抽出
def add_pnvalue(diclist_old):
    diclist_new = []
    for word in diclist_old:
        baseword = word['BaseForm']
        # PNTableに存在していればその単語の極性値を追加   
        if baseword in pn_dict:
            pn = float(pn_dict[baseword])
            
        else:
            pn = 'notfound'
        word['PN'] = pn
        diclist_new.append(word)
    return(diclist_new)

wordlist = get_diclist(file)
pn_list = add_pnvalue(wordlist)

print(pn_list)

[{'BaseForm': '*', 'PN': 'notfound'}, {'BaseForm': '*', 'PN': 'notfound'}, {'BaseForm': '法律', 'PN': -0.8285049999999999}, {'BaseForm': '知識', 'PN': -0.12055899999999999}, {'BaseForm': 'が', 'PN': 'notfound'}, {'BaseForm': 'ない', 'PN': -0.9999969999999999}, {'BaseForm': '方', 'PN': -0.9399870000000001}, {'BaseForm': 'が', 'PN': 'notfound'}, {'BaseForm': '頑張る', 'PN': 0.9817040000000001}, {'BaseForm': 'て', 'PN': 'notfound'}, {'BaseForm': '自分', 'PN': 'notfound'}, {'BaseForm': 'なり', 'PN': 'notfound'}, {'BaseForm': 'に', 'PN': 'notfound'}, {'BaseForm': '書く', 'PN': 'notfound'}, {'BaseForm': 'た', 'PN': 'notfound'}, {'BaseForm': 'という', 'PN': 'notfound'}, {'BaseForm': '点', 'PN': -0.650027}, {'BaseForm': 'は', 'PN': 'notfound'}, {'BaseForm': '評価', 'PN': -0.42166400000000004}, {'BaseForm': 'できる', 'PN': 'notfound'}, {'BaseForm': '。', 'PN': 'notfound'}, {'BaseForm': 'ただし', 'PN': 'notfound'}, {'BaseForm': '中身', 'PN': -0.592076}, {'BaseForm': 'は', 'PN': 'notfound'}, {'BaseForm': 'つけ', 'PN': 'notfound'}, {'Ba

In [0]:
list = [dic["PN"] for dic in pn_list if dic["PN"] != "notfound"]

In [0]:
def to_words(raw_data):
    
    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z]", " ",raw_data) 
    words = letters_only.lower().split()
    
    # ストップワードを削除します　https://qiita.com/Hironsan/items/2466fe0f344115aff177
    #stops = set(stopwords.words("english"))  
    #meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)]
    #return( " ".join(meaningful_words)) 
    return words

cleansed_impression = tweetData['text'].apply(lambda x: to_words(x))


all_text = ' '.join(cleansed_impression)
words = all_text.split()